Add Geospatial latitude and longitude data for a given postcode to the Postcode dataframe

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Store the required Postcode web page url 
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Use the BeautifulSoup package for processing the web page - set the returned data to be in an lxml format
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

# Scrape the Postcode table on the web page
web_table = soup.find('table',{'class':'wikitable sortable'})

# Load the contents of the Postcode table into a text array, use the strip method to remove unwanted characters from the dataset 
table_rows = []
# Loop through the contents of table based on tr and td tags
for table_row in web_table.find_all('tr'):
    table_data = table_row.find_all('td')
    # Convert each table row element to text, strip out unwanted characters and store in a text array
    table_rows.append([i.text.strip() for i in table_data]) 
    
# Isolate the Postcode table column headers (th - table header) and use the strip method to remove unwanted characters from the dataset
table_header = web_table.find_all('th')
# Convert each header list element to text and store in table_header 
table_header = [c.text for c in table_header]
# Loop through each of the characters in the string and use the strip method to remove unwanted characters such as '\n' new line escape codes
table_header = [i.strip() for i in table_header]
#print(table_header)

# Load the table rows and column headers into a dataframe
pdf = pd.DataFrame(data = table_rows, columns = table_header)
#pdf.head()

# Remove the empty top row from the dataframe and reset the index
pdf = pdf.drop([0])
pdf = pdf.reset_index(drop = True)
#pdf.head()

# Only include Boroughs in the dataframe that are assigned a Postcode - Ignore cells with a Borough that are 'Not assigned'
pdf = pdf[pdf.Borough != 'Not assigned']

# Sort the values in the dataframe
pdf.sort_values_by = ['Postcode', 'Borough', 'Neighborhood']

# For a given Postcode that has more than one assigned Neighbourhood, join the cells to form just one Postcode cell separated by a comma
pdf = pdf.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

# If a cell has a Borough but 'Not assigned' to a Neighborhood, then make the Neighborhood the same as the Borough
pdf.loc[pdf.Neighborhood == 'Not assigned', 'Neighborhood'] = pdf.Borough
pdf

pdf.shape

# Add Geospatial latitude and longitude data for a given postcode to the Postcode dataframe

# Locate the required geospatial data for a given postcode
geo_link = 'http://cocl.us/Geospatial_data'
geo_data = requests.get(geo_link)

# Strip out unwanted characters from the geospatial data and form into rows
geo_data = geo_data.text.strip('\n')
geo_data = geo_data.split('\r')

# Loop through each row in the geospatial data and strip out 'new line' esacpe codes
geo_data = [c.strip('\n') for c in geo_data]

# Split each geospatial data row into columns
geo_data = [i.split(',') for i in geo_data]

# Construct the Coordinates dataframe from the prepared geospatial data set
cdf = pd.DataFrame(data = geo_data, columns = geo_data[0])

# From the Coordinates dataframe, remove the top row, column headers, from the full data set and reset the index
cdf = cdf.drop([0])
cdf = cdf.reset_index(drop = True)

# Append Latitude and Longitude columns to the Postcode dataframe and initiate with 'null' data
pdf.insert(3, 'Latitude', '')
pdf.insert(4, 'Longitude', '')

# Where Postcode and Coordinate dataframe postcodes match, update the Postcode dataframe Latitude and Longitude values
# Loop through all the postcodes in the Postcode dataframe
for i in pdf['Postcode']:
    # Loop through all the postcodes in the Coordinates dataframe
    for c in cdf['Postal Code']:
        # Identify matching postcode values in the Postcode and Coordinates dataframes
        if i == c:
            # Based on matching postcode values, update the Postcode dataframe with the associated Latitude and Longitude values
            pdf.loc[pdf['Postcode'] == i, 'Latitude'] = cdf.loc[cdf['Postal Code'] == c, 'Latitude']
            pdf.loc[pdf['Postcode'] == i, 'Longitude'] = cdf.loc[cdf['Postal Code'] == c, 'Longitude']
            
pdf

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:44: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
5,M1J,Scarborough,Scarborough Village,43.7447342,-79.2394761
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279292,-79.2620294
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111117,-79.2845772
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.2394761
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.2648481
